# Dataset Creation

## QWK Dataset 

First, the QWK dataset is created that used the 5-fold cross validation. 

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict

df = pd.read_excel("../data/training_set_rel3.xlsx", index_col="essay_id")
df = df[df["domain1_score"].notnull()]

# load the folds from the txt files 
folds = []
for i in range(5):
    dataset = {"train": [], "dev": [], "test": []}
    for split in dataset.keys():
        ids = []
        with open(f"../data/splits/fold_{i}/{split}_ids.txt", "r") as f:
            for line in f:
                ids.append(int(line.strip()))
        if 10534 in ids:  # this line was only containing an essay 
            ids.remove(10534)
        dataset[split] = df.loc[ids]
    folds.append(dataset)

fold_datasets = []

for i in range(5):
    dataset = {"train": [], "dev": [], "test": []}
    for split in dataset.keys():
        dataset[split] = Dataset.from_pandas(folds[i][split], split=split)
    fold_datasets.append(DatasetDict(dataset))

folds = DatasetDict({"fold_{}".format(i): fold_datasets[i] for i in range(5)})

folds.save_to_disk("./data/datasets/folds")